In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#import mlcrate as mlc
from pathlib import Path

MODEL =  'distilbert-base-uncased'
dataPath = "C:/Users/shash/Documents/MScA/Courses/Summer 2021/Advanced Machine Learning/Session 6/Toxic Comment Data/jigsaw-toxic-comment-classification-challenge/"

model_path= Path(os.getcwd()).parent.parent/"NewNLP"/MODEL
#print("Data: ",dataPath,"\nModel: ",model_path)
reviews = pd.read_csv(dataPath + 'tc_train.csv', index_col=0)
reviews.head()



,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [2]:
## Test data
reviews_test = pd.read_csv(dataPath + 'tc_test.csv', index_col=0)
reviews_test.head()

,comment_text
id,
00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
0000247867823ef7,== From RfC == \n\n The title is fine as it is...
00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
00017563c3f7919a,":If you have a look back at the source, the in..."
00017695ad8997eb,I don't anonymously edit articles at all.


In [3]:
types = list(reviews)[1:]
print(types)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [4]:
print('Import Transformers')
from transformers import AutoTokenizer, TFAutoModel, logging
folder = os.fspath(model_path)

Import Transformers


In [5]:
transformer = TFAutoModel.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
print('Transformer: ',transformer)
print('Tokenizer: ',tokenizer)
transformer.save_pretrained(folder)
tokenizer.save_pretrained(folder)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Transformer:  <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertModel object at 0x0000027BFFC00B48>
Tokenizer:  PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


('C:\\Users\\shash\\Documents\\MScA\\Courses\\Summer 2021\\Advanced Machine Learning\\NewNLP\\distilbert-base-uncased\\tokenizer_config.json',
 'C:\\Users\\shash\\Documents\\MScA\\Courses\\Summer 2021\\Advanced Machine Learning\\NewNLP\\distilbert-base-uncased\\special_tokens_map.json',
 'C:\\Users\\shash\\Documents\\MScA\\Courses\\Summer 2021\\Advanced Machine Learning\\NewNLP\\distilbert-base-uncased\\vocab.txt',
 'C:\\Users\\shash\\Documents\\MScA\\Courses\\Summer 2021\\Advanced Machine Learning\\NewNLP\\distilbert-base-uncased\\added_tokens.json',
 'C:\\Users\\shash\\Documents\\MScA\\Courses\\Summer 2021\\Advanced Machine Learning\\NewNLP\\distilbert-base-uncased\\tokenizer.json')

In [9]:
logging.set_verbosity(logging.ERROR)  # suppress warnings, output errors only
transformer = TFAutoModel.from_pretrained(folder)
tokenizer = AutoTokenizer.from_pretrained(folder)
print(transformer)
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='C:\Users\shash\Documents\MScA\Courses\Summer 2021\Advanced Machine Learning\NewNLP\distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [6]:
MAX_LEN = 64
def encode_data(data, max_len):
    enc_data = tokenizer(data,max_length=max_len, return_token_type_ids=False,
                         padding=True, truncation=True)
    return [np.array(enc_data[k]) for k in ['input_ids','attention_mask']]

In [7]:
data = encode_data(reviews.comment_text.to_list(), max_len=MAX_LEN)
print('Shape of the encoded data: ',data[0].shape)

## Test data
data_test = encode_data(reviews_test.comment_text.to_list(), max_len=MAX_LEN)
print('Shape of the encoded data: ',data_test[0].shape)


Shape of the encoded data:  (223549, 64)
Shape of the encoded data:  (89186, 64)


In [8]:
import tensorflow as tf
from tensorflow.keras import Model, Input 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score

def get_model(transformer):
    input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    input_att_mask = Input(shape=(MAX_LEN,), dtype=tf.int8, name="input_att_mask")
    transformer.trainable = True
    x = transformer([input_word_ids,input_att_mask])[0][:, 0, :]  
    out = Dense(6, activation='softmax',name = 'custom_dense')(x)
    model = Model(inputs=[input_word_ids,input_att_mask], outputs=out)
    model.compile(loss="categorical_crossentropy", 
                  optimizer=Adam(learning_rate=5e-6))
    return model

### Split the data into train and test parts

In [9]:
id_train,id_test,att_train,att_test,y_train,y_val = train_test_split(data[0], data[1],
                reviews[types].to_numpy(), test_size=0.2,  random_state=0)
X_train, X_val = [id_train,att_train],[id_test,att_test]

In [15]:
print('Shapes for train: ',id_train.shape, y_train.shape)
print('Shapes for validation: ',id_test.shape,y_val.shape)

Shapes for train:  (178839, 64) (178839, 6)
Shapes for validation:  (44710, 64) (44710, 6)


In [18]:
idx = np.random.randint(id_train.shape[0], size=2000)
X_train_subsample=[X_train[0][idx,:],X_train[1][idx,:]]
y_train_subsample=y_train[idx]

In [19]:
cls = transformer.predict(X_train_subsample,batch_size=32)[0][:,0,:].astype(np.float32)
print(cls.shape)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
(2000, 768)


In [21]:
best_model_path = 'best_model_head.h5'
BATCH_SIZE = 128

# save the best model to a file
chp = ModelCheckpoint(best_model_path,save_best_only=True,save_weights_only=True)
model = get_model(transformer)
print('Model: classifier head traning only: \n',model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_att_mask (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    input_word_ids[0][0]             
                                                                 input_att_mask[0][0]             
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           tf_distil_bert_model_1[0][0] 

In [ ]:
hist = model.fit(X_train, y_train,validation_data=(X_val, y_val),
                 epochs=1, batch_size=BATCH_SIZE, shuffle=True, 
                 verbose=2,callbacks=[chp])